In [1]:
# libraries
import os
import json
import numpy as np
import pandas as pd
from extra_files import helper as hp
from imageio import imwrite, imread
from skimage.transform import resize

In [2]:
# File paths
data_path = '/home/aldo/Documents/data-cic/'
preprocess_path = data_path + 'preprocess_data'

## Training SSDLite300

In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssdlite320_mobilenetv2 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from extra_files.f1_callback import F1_callback as f1_call
%matplotlib inline

Using TensorFlow backend.
/home/aldo/Documents/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


### Parameters (original SSD300 architecture)

In [4]:
## Parameteres needed for ssd_300() and SSDInputEncoder()

img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [1., 1., 1.] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
divide_by_stddev = [127.5, 127.5, 127.5]
swap_channels = False # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 1 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
new_scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
scales = new_scales
aspect_ratios = [[1.0, 0.5, 2.0/3.0],
                 [1.0, 0.5, 2.0/3.0, 1.0/3.0, 3.0/4.0],
                 [1.0, 2.0, 0.5, 2.0/3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 2.0/3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [16, 30, 60, 100, 150, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True
list_alpha = [1.0, 0.35]
list_expansion = [1]

## Data generator for the training

In [ ]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset_pascal = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset_pascal = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
images_dir = data_path + 'pascal_dataset'

# Ground truth
train_labels_filename = preprocess_path + '/pascal_train.csv'
val_labels_filename   = preprocess_path + '/pascal_val.csv'

train_dataset_pascal.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset_pascal.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Get the number of samples in the training and validations datasets.
train_dataset_size_pascal = train_dataset_pascal.get_dataset_size()
val_dataset_size_pascal   = val_dataset_pascal.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size_pascal))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size_pascal))

# 3: Set the batch size.
batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [(18, 18),
                   (9, 9),
                   (5, 5),
                   (3, 3),
                   (2, 2),
                   (1, 1)]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.6,
                                    neg_iou_limit=0.3,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator_pascal = train_dataset_pascal.generate(batch_size=batch_size,
                                              shuffle=True,
                                              transformations=[ssd_data_augmentation],
                                              label_encoder=ssd_input_encoder,
                                              returns={'processed_images',
                                                       'encoded_labels'},
                                              keep_images_without_gt=False)

val_generator_pascal = val_dataset_pascal.generate(batch_size=batch_size,
                                          shuffle=False,
                                          transformations=[convert_to_3_channels,
                                                           resize],
                                          label_encoder=ssd_input_encoder,
                                          returns={'processed_images',
                                                   'encoded_labels'},
                                          keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size_pascal = train_dataset_pascal.get_dataset_size()
val_dataset_size_pascal   = val_dataset_pascal.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size_pascal))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size_pascal))

Loading images into memory: 100%|██████████| 2097/2097 [00:08<00:00, 235.50it/s]
Number of images in the training dataset:	  6469
Number of images in the validation dataset:	  2097
Number of images in the training dataset:	  6469
Number of images in the validation dataset:	  2097


In [ ]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset_cic = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset_cic = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
images_dir = data_path + 'images'

# Ground truth
train_labels_filename = preprocess_path + '/cic_train.csv'
val_labels_filename   = preprocess_path + '/cic_val.csv'

train_dataset_cic.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset_cic.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Get the number of samples in the training and validations datasets.
train_dataset_size_cic = train_dataset_cic.get_dataset_size()
val_dataset_size_cic   = val_dataset_cic.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size_cic))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size_cic))

# 3: Set the batch size.
batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [(18, 18),
                   (9, 9),
                   (5, 5),
                   (3, 3),
                   (2, 2),
                   (1, 1)]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.6,
                                    neg_iou_limit=0.3,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator_cic = train_dataset_cic.generate(batch_size=batch_size,
                                            shuffle=True,
                                            transformations=[ssd_data_augmentation],
                                            label_encoder=ssd_input_encoder,
                                            returns={'processed_images',
                                                     'encoded_labels'},
                                            keep_images_without_gt=False)

val_generator_cic = val_dataset_cic.generate(batch_size=batch_size,
                                        shuffle=False,
                                        transformations=[convert_to_3_channels,
                                                         resize],
                                        label_encoder=ssd_input_encoder,
                                        returns={'processed_images',
                                                 'encoded_labels'},
                                        keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size_cic = train_dataset_cic.get_dataset_size()
val_dataset_size_cic   = val_dataset_cic.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size_cic))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size_cic))

Loading images into memory: 100%|██████████| 45/45 [00:01<00:00, 27.58it/s]
Number of images in the training dataset:	   210
Number of images in the validation dataset:	    45
Number of images in the training dataset:	   210
Number of images in the validation dataset:	    45


In [ ]:
label_val = np.load('../data-cic/preprocess_data/label_val.npy')
val_images_300 = np.load('../data-cic/preprocess_data/images_val_300x300.npy')

In [ ]:
def return_callbacks(path_weights, name_weights, path_logger, path_tensorboard, path_f1, save_f1):
    
    # Define model callbacks.
    # TODO: Set the filepath under which you want to save the model.
    model_checkpoint = ModelCheckpoint(filepath=path_weights + name_weights,
                                       monitor='val_loss',
                                       verbose=1,
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='auto',
                                       period=1)

    csv_logger = CSVLogger(filename= path_logger,
                           separator=',',
                           append=True)

    reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                             factor=0.5,
                                             patience=10,
                                             verbose=1,
                                             min_delta=0.001,
                                             cooldown=0,
                                             min_lr=0.000001)
    
    tbCallBack = TensorBoard(log_dir=path_tensorboard, 
                                       histogram_freq=0, 
                                       write_graph=False, 
                                       write_images=False,
                                       update_freq='epoch')


    f1_callback = f1_call(0.20, 
                          0.45, 
                          200, 
                          normalize_coords, 
                          img_height, 
                          img_width, 
                          val_images_300, 
                          label_val, (1, 2006, 14),
                          path_f1,
                          save_f1)

    callbacks = [model_checkpoint,
                 csv_logger,
                 reduce_learning_rate,
                 f1_callback,
                 tbCallBack]
    return callbacks

## Loop to train different models

In [ ]:
for alpha in list_alpha:
    for expansion in list_expansion:
        # 1: Build the Keras model.

        K.clear_session() # Clear previous models from memory.

        model = ssd_300(image_size=(img_height, img_width, img_channels),
                        n_classes=n_classes,
                        alpha=alpha,
                        expansion=expansion,
                        mode='training',
                        scales=scales,
                        aspect_ratios_per_layer=aspect_ratios,
                        two_boxes_for_ar1=two_boxes_for_ar1,
                        steps=steps,
                        offsets=offsets,
                        clip_boxes=clip_boxes,
                        variances=variances,
                        normalize_coords=normalize_coords,
                        subtract_mean=mean_color,
                        divide_by_stddev=divide_by_stddev,
                        swap_channels=swap_channels)

        print('\n---->Training model with alpha', alpha, 'and expansion', expansion, 'with pascal')
        print('Number of parameters:', model.count_params())
        
        # Train over pascal
        adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
        model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
        
        # set callbacks
        str_alpha = str(alpha)
        str_exp = str(expansion)
        callbacks_pascal =  return_callbacks('/home/aldo/Documents/weights/models/last_models/', 
                              '2pos_4_alpha_1.5_pascal_' + str_alpha + '_exp_' + str_exp + '.h5',
                              data_path + 'history/last_models/2pos_4_alpha_1.5_pascal_alpha_' + str_alpha + '_exp_' + str_exp + '.csv',
                              data_path + 'history/last_models/tensorboard/pascal/2pos_4_alpha_1.5' + str_alpha + '_exp_' + str_exp +'/pascal_' + str_alpha + '_exp_' + str_exp,
                              data_path + 'history/last_models/2pos_4_alpha_1.5_pascal_f1_' + str_alpha + '_exp_' + str_exp + '.csv',
                              '/home/aldo/Documents/weights/models/last_models/' + '2pos_4_alpha_1.5_pascal_f1_' + str_alpha + '_exp_' + str_exp + '.h5' )
        # If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
        initial_epoch   = 0
        final_epoch     = 20
        steps_per_epoch = 400

        history = model.fit_generator(generator=train_generator_pascal,
                                      steps_per_epoch=steps_per_epoch,
                                      epochs=final_epoch,
                                      callbacks=callbacks_pascal,
                                      validation_data=val_generator_pascal,
                                      validation_steps=ceil(val_dataset_size_pascal/batch_size),
                                      initial_epoch=initial_epoch)
            
        print('\n---->Training model with alpha', alpha, 'and expansion', expansion, 'with cic')
        # Train over cic
        # Reset learning rate to 0.0005
        adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
        model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
        
        callbacks_cic =  return_callbacks('/home/aldo/Documents/weights/models/last_models/', 
                              '2pos_4_alpha_1.5_pascal_cic_' + str_alpha + '_exp_' + str_exp + '.h5',
                              data_path + 'history/last_models/2pos_4_alpha_1.5_pascal_cic_alpha_' + str_alpha + '_exp_' + str_exp + '.csv',
                              data_path + 'history/last_models/tensorboard/cic/2pos_4_alpha_1.5' + str_alpha + '_exp_' + str_exp +'/cic' + str_alpha + '_exp_' + str_exp,                              
                              data_path + 'history/last_models/2pos_4_alpha_1.5_pascal_cic_f1_' + str_alpha + '_exp_' + str_exp + '.csv',
                              '/home/aldo/Documents/weights/models/last_models/' + '2pos_4_alpha_1.5_pascal_cic_f1_' + str_alpha + '_exp_' + str_exp + '.h5')
        
        initial_epoch   = 0
        final_epoch     = 20
        steps_per_epoch = 15

        history = model.fit_generator(generator=train_generator_cic,
                                      steps_per_epoch=steps_per_epoch,
                                      epochs=final_epoch,
                                      callbacks=callbacks_cic,
                                      validation_data=val_generator_cic,
                                      validation_steps=ceil(val_dataset_size_cic/batch_size),
                                      initial_epoch=initial_epoch)


---->Training model with alpha 1.0 and expansion 1 with pascal
Number of parameters: 2618612
Epoch 1/20
400/400 [==============================] - 236s 590ms/step - loss: 8.1856 - val_loss: 6.7928

Epoch 00001: val_loss improved from inf to 6.79276, saving model to /home/aldo/Documents/weights/models/last_models/2pos_4_alpha_1.5_pascal_1.0_exp_1.h5
F1 score: 0.2213778944620361
Improve F1 score from -inf to 0.2213778944620361
Epoch 2/20
400/400 [==============================] - 228s 570ms/step - loss: 4.1244 - val_loss: 4.0912

Epoch 00002: val_loss improved from 6.79276 to 4.09125, saving model to /home/aldo/Documents/weights/models/last_models/2pos_4_alpha_1.5_pascal_1.0_exp_1.h5
F1 score: 0.25732879272339787
Improve F1 score from 0.2213778944620361 to 0.25732879272339787
Epoch 3/20
400/400 [==============================] - 226s 565ms/step - loss: 3.7622 - val_loss: 3.7950

Epoch 00003: val_loss improved from 4.09125 to 3.79505, saving model to /home/aldo/Documents/weights/models/l